#### <font color=red>2021/2/9 本版本<br>(1)部分底层数据重清洗，做到brand和goods中的一致;<br>(2)数据增强以后测试少样本品牌的ner预测;<br><br>下次版本<br>(1)加入天猫/淘宝/京东等第三方统计好的我们没有的品牌；<br>(2)确认所有品牌是真实有效的，进行人工核检;<br>(3)如果上面版本测试的数据增强的效果可以的话，则购买第三方完整的品牌数据后做数据增强;<br><br>下下次版本<br>(1)加入品类标签,CNT-B,CNT-I;<br>(2)制作品牌+国家的映射表，直接自动化处理dupont的国家的5个master字段；

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sqlalchemy import create_engine
import time,gc,os,re,sys,re,jieba,os,math
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
def check_cols(df):
    cols=df.columns.tolist()
    leng=int(len(cols)/5)+1
    for i in range(leng):
        tmp=df.iloc[:5,i*6:(i+1)*6]
        print(tmp.head())
        print('-'*30)

def check_miss_ratio(df):
    missdf=pd.isnull(df).sum().reset_index(name='miss_cnt')
    missdf.columns=['col','miss_cnt']
    missdf['miss_ratio']=missdf['miss_cnt']/df.shape[0]
    print(missdf.head(20))
    return missdf

#### <font color=red>20210209修改：不做补充缺失值，直接使用所有的字段

In [3]:
# great_dic={'乐玩美研':'BCL','欧莱雅':'巴黎欧莱雅',"L'OREAL":"L'OREALPARIS","自然乐园":"自然共和国","乔治阿玛尼":"阿玛尼",
#            "GIORGIOARMANI":"ARMANI","AEKYUNG":"AEKYUNGAGE'S","伊诗露":"伊特露"}

In [4]:
def auto_mark_1(s,k):
    leng=len(s)
    lst=[]
    
    summ=0
    for i in range(leng):
        word=s[i]
        if word!=k[0]:
            lst.append('O')
            summ+=1
        elif word==k[0]:
            lst.append('BRD-B')
            for j in range(i+1,leng):
                word1=s[j]
                if word1 in k:
                    lst.append('BRD-I')
                    summ+=1
                else:
                    break
    return s


def get_key_brd(k):
    if type(k)=='float':
        k=str(int(k))
    leng=len(k)
    lst=['BRD-B']
    for i in range(leng-1):
        lst.append('BRD-I')
    return lst


def auto_mark_2(s,k):
    klist=get_key_brd(k)

    s_leng=len(s)
    
    final_s_2=[]
    if k in s:
        s=s.replace(k,'#BRD#')
        lst=s.split('#BRD#')
        lst1=lst[0]
        if lst1=='':
            final_s_2.extend(klist)
            for i in range(1,len(lst)):
                if lst[i]:
                    new_list=[x for x in lst[i]]
                    final_s_2.extend('O'*len(new_list))
                    if i!=len(lst)-1:
                        final_s_2.extend(klist)
                    else:
                        continue

        else:
            for i in range(len(lst)):
                if lst[i]:
                    new_list=[x for x in lst[i]]
                    final_s_2.extend('O'*len(new_list))
                    if i!=len(lst)-1:
                        final_s_2.extend(klist)
                    else:
                        continue
    return ','.join(final_s_2)


def auto_mark_3(s,k):

    klist=get_key_brd(k)
    
    lst=[]
    if s==k:
        lst.extend(klist)
    else:
        ss=s.split(k)
        for i in range(len(ss)):
            if ss[i]=='':
                if i!=len(ss)-1:
                     lst.extend(klist)
            else:
                lst.extend(['O' for x in ss[i]])
                if i!=len(ss)-1:
                    lst.extend(klist)
    leng=len(lst)
    return ','.join(lst),leng

def auto_mark_33(s,k):
    if type(k)=='float':
        k=str(int(k))
        
    klist=get_key_brd(k)
    
    lst=[]
    if s==k:
        lst.extend(klist)
    else:
        ss=s.split(k)
        for i in range(len(ss)):
            if ss[i]=='':
                if i!=len(ss)-1:
                     lst.extend(klist)
            else:
                lst.extend(['O' for x in ss[i]])
                if i!=len(ss)-1:
                    lst.extend(klist)
    return ','.join(lst)

In [5]:
df=pd.read_csv('data_1.csv')
print(df.shape)

(4519931, 12)


In [6]:
df.loc[df['brand_dig'].notnull(),'brand_dig']=df[df['brand_dig'].notnull()]['brand_dig'].apply(lambda x:str(int(x)))

In [7]:
def further_clean(v):
    v=str(v).replace(' ','')
    v=v.upper()
    pattern='[^0-9A-Z\u4e00-\u9fa5]'
    v=re.sub(pattern,'',v)
    return v

In [8]:
str_cols=['brand', 'brand_cn', 'brand_cn_fill', 'brand_dig', 'brand_en', 'brand_en_fill', 'company_cn', 'company_en', 'desc']

for col in str_cols:
    df.loc[(df[col].notnull())&(df[col]!=''),col]=df[(df[col].notnull())&(df[col]!='')][col].apply(lambda x:further_clean(x))
    gc.collect()

In [9]:
new_brand_col_dict={'赫妍':'赫拉', '悦诗风吟':'风吟', '乔治阿玛尼':'阿玛尼', '伊蒂之屋':'爱丽小屋', '纪梵希':'紀梵希',
                    '珂莱欧':'可莱欧', '魅可':'MAC', '玛丽黛佳':'玛丽佳人', '资生堂':'資生堂',
                    '倩碧':'倩倩', '芭比波朗':'波比布朗', '兰蔻':'兰KOU', '菲诗小铺':'菲斯小铺',
                    "SWEETCOLOR":"SWEETCOLO",
                    "ILLAMASQUA":"ILLAMASUQA",
                    "姬芮":"ZA", "HOLIKAHOLIKA":"HOLIKA", "三熹玉":"3CE", "雪詩緹":"雪诗缇", "THREE":"777",
                    "曼秀雷敦":"曼秀雷", "SMITHCHU":"禇铁匠","DRECODE":"DRECDE"}

In [10]:
df.loc[df['brand_cn'].notnull(),'brand_cn_simple']=df[df['brand_cn'].notnull()]['brand_cn'].apply(lambda x:new_brand_col_dict[x] if x in new_brand_col_dict else np.nan)
df.loc[(df['brand_cn_simple'].isnull())&(df['brand_en'].notnull()),'brand_cn_simple']=df[(df['brand_cn_simple'].isnull())&(df['brand_en'].notnull())]['brand_en'].apply(lambda x:new_brand_col_dict[x] if x in new_brand_col_dict else np.nan)

In [11]:
for col in str_cols:
    df.loc[df[col]=='',col]=np.nan
    df.loc[df[col]=='nan',col]=np.nan
    gc.collect()

In [12]:
dff=df[df['desc'].notnull()]
print(dff.shape)

(4519472, 13)


In [13]:
test=dff[(dff['brand_cn'].isnull())&(dff['brand_cn_fill'].isnull())&(dff['brand_dig'].isnull())&(dff['brand_en'].isnull())&(dff['brand_en_fill'].isnull())&(dff['company_cn'].isnull())&(dff['company_en'].isnull())&(dff['brand_cn_simple'].isnull())]
print(test.shape)

(299402, 13)


In [14]:
train=dff[~dff['id'].isin(set(test['id']))]
print(train.shape)

(4220070, 13)


In [15]:
print(len(train)/len(dff))

0.9337528808674995


In [16]:
train=train.reset_index(drop=True)
train['leng']=train['desc'].apply(len)
gc.collect()

7

In [17]:
test=test.reset_index(drop=True)
test['leng']=test['desc'].apply(len)
gc.collect()

7

In [18]:
# for col in ['brand_cn', 'brand_cn_fill', 'brand_dig', 'brand_en', 'brand_en_fill', 'company_cn', 'company_en', 'brand_cn_simple']:
#     print(col)
#     train[col]=train[col].astype(str)

In [19]:
# check1=train[train['brand_cn']=='']
# print(check1.shape)
# check2=train[train['brand_cn'].isnull()]
# print(check2.shape)
# check3=train[train['brand_cn']=='nan']
# print(check3.shape)
# gc.collect()

In [20]:
train['brand_cn']=train['brand_cn'].astype(str)

In [21]:
def check_col(tmp,col):
    check1=tmp[tmp[col].isnull()]
    print(check1.shape)
    check2=tmp[tmp[col]=='']
    print(check2.shape)
    check3=tmp[tmp[col]=='nan']
    print(check3.shape)

In [22]:
check_col(train,'brand_cn')

(0, 14)
(0, 14)
(991616, 14)


In [23]:
print(pd.isnull(train[['id','brand_cn']]).sum())

id          0
brand_cn    0
dtype: int64


In [24]:
train['brand_cn_ner']=train[['desc','brand_cn']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)

In [25]:
train['brand_cn_fill']=train['brand_cn_fill'].astype(str)

In [26]:
check_col(train,'brand_cn_fill')

(0, 15)
(0, 15)
(1410252, 15)


In [27]:
print(pd.isnull(train[['id','brand_cn_fill']]).sum())

id               0
brand_cn_fill    0
dtype: int64


In [29]:
train['brand_cn_fill_ner']=train[['desc','brand_cn_fill']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)

In [30]:
train['brand_en']=train['brand_en'].astype(str)
train['brand_en_ner']=train[['desc','brand_en']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)

In [31]:
train['brand_en_fill']=train['brand_en_fill'].astype(str)
train['brand_en_fill_ner']=train[['desc','brand_en_fill']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)
gc.collect()

In [33]:
train['company_cn']=train['company_cn'].astype(str)
train['company_cn_ner']=train[['desc','company_cn']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)

In [34]:
train['company_en']=train['company_en'].astype(str)
train['company_en_ner']=train[['desc','company_en']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)

In [35]:
train['brand_dig']=train['brand_dig'].astype(str)
train['brand_dig_ner']=train[['desc','brand_dig']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)

In [36]:
train['brand_cn_simple']=train['brand_cn_simple'].astype(str)
train['brand_cn_simple_ner']=train[['desc','brand_cn_simple']].apply(lambda row:auto_mark_33(row[0],row[1]),raw=True,axis=1)

In [37]:
gc.collect()

28

In [38]:
train[['id','desc','brand_cn_simple_ner','brand_cn_ner']].head()

,id,desc,brand_cn_simple_ner,brand_cn_ner
0,10023690981294,943934脸部温和眼部无刺激清洁女敏感肌氨基酸卸妆水280ML脸部任何肤质国产,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,10313567492,超值推荐美国品牌大号眼影刷和小号眼影刷柔软小马毛,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,10315667189,冲四冠好用的环保ECOTOOLS化妆刷4件套天然竹子柄送刷包,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,10678807752,7折特价PHILOSOPHY半圆型便携版散粉腮红刷可放置粉盒内,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","O,O,O,O,BRD-B,BRD-I,BRD-I,BRD-I,BRD-I,BRD-I,BR..."
4,11185298785,1611面若桃花气垫BB霜12G2盒送替换装底妆自然色裸妆粉饼遮瑕保湿隔离霜粉底液,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [39]:
gc.collect()

10

In [40]:
def get_final_ner(v1,v2,v3,v4,v5,v6,v7,v8,leng):
    v1=list(v1.split(','))
    v2=list(v2.split(','))
    v3=list(v3.split(','))
    v4=list(v4.split(','))
    v5=list(v5.split(','))
    v6=list(v6.split(','))
    v7=list(v7.split(','))
    v8=list(v8.split(','))
    
    if len(v1)==len(v2)==len(v3)==leng:
        new_lst=[]
        for i in range(leng):
            v11=v1[i]
            v22=v2[i]
            v33=v3[i]
            v44=v4[i]
            v55=v5[i]
            v66=v6[i]
            v77=v7[i]
            v88=v8[i]
            if 'BRD-I' in [v11,v22,v33,v44,v55,v66,v77,v88]: #注意里面有顺序关系，比如悦诗风吟和风吟
                new_lst.append('BRD-I')
            elif 'BRD-B' in [v11,v22,v33,v44,v55,v66,v77,v88]:
                new_lst.append('BRD-B')
            else:
                new_lst.append('O')
        return ','.join(new_lst)
    else:
        return 'wrong'

In [41]:
train['final_ner']=list(map(lambda a,b,c,d,e,f,g,h,leng:get_final_ner(a,b,c,d,e,f,g,h,leng),train['brand_cn_ner'],train['brand_en_ner'],train['company_cn_ner'],train['company_en_ner'],train['brand_cn_simple_ner'],train['brand_cn_fill_ner'],train['brand_en_fill_ner'],train['brand_dig_ner'],train['leng']))
gc.collect()

7

In [42]:
check=train[train['final_ner']=='wrong']
print(check.shape)
if len(check)>0:
    check.to_csv('ner_wrong_check.csv',index=False,encoding='utf-8-sig')
gc.collect()

(0, 23)


0

In [43]:
train_df=train[train['final_ner']!='wrong']
print(len(train_df)/len(train))
train_df.to_csv('train_ner.csv',index=False,encoding='utf-8-sig')
gc.collect()

1.0


0

In [44]:
test_df=test.copy()
test_df.to_csv('test_ner.csv',index=False,encoding='utf-8-sig')
gc.collect()

0

### <font color=red>检查train_ner

#### <font color=blue>(1)检查整体

In [45]:
print('开始检查整体')
for i in range(1000,1020,1):
    print(train_df['desc'][i])
    print(train_df['final_ner'][i])
    print(train_df['brand_cn'][i])
    print(train_df['brand_en'][i])
    print(train_df['brand'][i])
    print(train_df['brand_dig'][i])
    print('-'*40)

开始检查整体
预订韩国免税店代购DARPHIN朵法花梨木按摩洁面膏40ML
O,O,O,O,O,O,O,O,O,BRD-B,BRD-I,BRD-I,BRD-I,BRD-I,BRD-I,BRD-I,O,O,O,O,O,O,O,O,O,O,O,O,O,O
迪梵
DARPHIN
2050
2050
----------------------------------------
WM不沾杯不脱色豆沙脏橘吃土色唇釉南瓜色唇漆防水唇彩买二送一
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
919
919
----------------------------------------
保湿绿茶薰衣草卸妆泡沫
O,O,O,O,O,O,O,O,O,O,O
nan
nan
2050
2050
----------------------------------------
1905面膜刷自制面膜扫纤维柔面膜软毛刷涂面膜工具刷
BRD-B,BRD-I,BRD-I,BRD-I,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
1905
1905
----------------------------------------
馥草堂美容院正品净颜控油洁面乳新包装带防伪包邮
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
2050
2050
----------------------------------------
美容仪专用导出水清洁卸妆水正品脸部温和控油去污保湿收缩毛孔
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
2050
2050
----------------------------------------
15色滋润唇彩口红盘裸色唇膏口红盒子板专业彩妆盘正品3色可选
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
919
919
----------------------------------

#### <font color=blue>(2)检查没有实体的数据

In [46]:
#真正需要检查的是,都是O的这样的情况,看看里面确实是没有关键字还是因为品牌的名字多写了一个字母或者少写了一个字母
def get_all_zero(v):
    v=v.split(',')
    s=0
    for w in v:
        if w=='O':
            s+=1
    if s==len(v):
        return 'all_zero'
    else:
        return 'have_key'

In [47]:
train_df['mark']=train_df['final_ner'].apply(get_all_zero)

In [48]:
check=train_df[train_df['mark']=='all_zero'].reset_index(drop=True)
print(check.shape)

(1189550, 24)


In [49]:
print('开始检查都是O的数据')
for i in range(100,200):
    print(check['desc'][i])
    print(check['final_ner'][i])
    print(check['brand_cn'][i])
    print(check['brand_en'][i])
    print('-'*40)
gc.collect()

开始检查都是O的数据
便携式伸缩唇刷热卖彩妆化妆工具适用唇彩唇膏唇蜜
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
天然环保竹子柄4支化妆刷套装腮红刷蜜粉刷眼影刷遮瑕刷包邮
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
正品包邮化妆师初学者10支经典化妆刷套刷彩妆工具全套化妆扫
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
指甲油按斤称厂家库存外贸尾货STOCK义乌小商品地摊2元店只批发
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
新款外贸热销款9支动物毛化妆刷套装便携款
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
大卷巾美容院必备全棉无纺布一次性洗面巾洁面巾美容巾化妆棉巾
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
魔幻彩色环保指甲油无毒无味孕妇儿童可用持久光泽易吸收美甲油
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
专业蘑菇刷刷桶便携式收纳包小刷桶
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
------------

0